# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Read in the CSV file
cities_df = pd.read_csv("../WeatherPy/output_data/cities.csv", encoding="utf-8")


cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Narsaq,60.92,-46.05,30.20,54,75,2.24,GL,1601464402
1,Manta,-0.95,-80.73,69.30,85,99,7.38,EC,1601464402
2,Longyearbyen,78.22,15.64,39.20,80,75,14.99,SJ,1601464402
3,Butaritari,3.07,172.79,82.08,76,1,13.56,KI,1601464403
4,Vila Franca do Campo,37.72,-25.43,68.00,64,40,8.05,PT,1601464403


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Store 'Lat' and 'Lng' into  locations 
locations = cities_df[["Lat", "Lng"]]


# Convert Humidity to float and store
humidity = cities_df["Humidity"].astype(float)

In [20]:
# Create a Heatmap layer - 100 intensity used as it is the max humidity
fig = gmaps.figure(zoom_level=1.8, center=(20,10))

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100,point_radius=2)

fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows with null values.

In [21]:
# Narrow down dataframe to fit weather conditions
# I chose, max temp lower than 90F and greater than 80F, zero cloudiness, and wind speed less than 5mph  
ideal_city_df = cities_df.loc[(cities_df["Max Temp"] > 80) & (cities_df["Max Temp"] < 90) & (cities_df["Cloudiness"] == 0) & (cities_df["Wind Speed"] < 5)]
ideal_city_df = ideal_city_df.dropna()
ideal_city_df.reset_index(drop=True)

ideal_city_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
287,Cabo San Lucas,22.89,-109.91,80.01,75,0,2.44,MX,1601464473
288,Dalbandin,28.89,64.41,85.96,14,0,0.51,PK,1601464281
447,Tabas,33.60,56.92,80.60,12,0,2.24,IR,1601464510


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [22]:
# Call my DataFrame hotel_df
hotel_df = ideal_city_df

# Add Hotel Name column to the DataFrame
hotel_df["Hotel Name"] = ""

# Find the first hotel for each city within 5000 meters of the coordinates

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Base url for get request
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request
    hotel_name = requests.get(base_url, params=params)
    

    # convert to json
    hotel_name = hotel_name.json()

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df    

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
287,Cabo San Lucas,22.89,-109.91,80.01,75,0,2.44,MX,1601464473,Welk Resorts Cabo San Lucas - Sirena del Mar
288,Dalbandin,28.89,64.41,85.96,14,0,0.51,PK,1601464281,Al-Dawood Hotel & Restaurant
447,Tabas,33.60,56.92,80.60,12,0,2.24,IR,1601464510,Amir hotel


In [23]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [26]:
# Add marker layer ontop of heat map
gmaps.figure(zoom_level=1.75, center=(20,10))
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure

fig

Figure(layout=FigureLayout(height='420px'))